<a href="https://colab.research.google.com/github/azzindani/03_Prompt_Engineering/blob/main/PE_EN_SmolLM2_1_7B_Instruct_Med_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Modules

In [1]:
!pip install -U -q bitsandbytes
!pip install -q optimum
!pip install -q auto-gptq

In [2]:
import torch

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  GenerationConfig,
  BitsAndBytesConfig,
  GPTQConfig
)

if torch.cuda.is_available():
  print('GPU is available!')
else:
  print('GPU is not available.')

GPU is available!


## Import Model

In [3]:
model_name = 'HuggingFaceTB/SmolLM2-1.7B-Instruct'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
'''bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  trust_remote_code = True
).to(device) #'''

"bnb_config = BitsAndBytesConfig(\n  load_in_4bit = True,\n  bnb_4bit_quant_type = 'nf4',\n  bnb_4bit_compute_dtype = torch.float16,\n  bnb_4bit_use_double_quant = True,\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n  model_name,\n  quantization_config = bnb_config,\n  trust_remote_code = True\n).to(device) #"

In [6]:
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Import Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

## Model Preview

In [8]:
def assistant(prompt):
  prompt_format = f"### Human: {prompt}\n### Assistant :"

  inputs = tokenizer(
    prompt_format,
    return_tensors = 'pt'
  ).to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [9]:
# Level 1: Basic Prompt
prompt = "What are the symptoms and treatments of diabetes?"
assistant(prompt)

### Human: What are the symptoms and treatments of diabetes?
### Assistant :
Diabetes is a chronic condition that affects how your body regulates blood sugar levels. Symptoms can vary depending on the type of diabetes, but common symptoms include increased thirst and urination, fatigue, blurred vision, and slow healing of cuts and wounds.

There are two main types of diabetes: Type 1 and Type 2. Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the cells in the pancreas that produce insulin. Type 2 diabetes is a metabolic disorder where the body becomes resistant to insulin or doesn't produce enough insulin.

Treatments for diabetes include lifestyle changes such as diet and exercise, as well as medication or insulin therapy. For Type 1 diabetes, patients require insulin therapy to manage their blood sugar levels. For Type 2 diabetes, lifestyle changes and oral medications are often used to manage blood sugar levels. In some cases, insulin the

In [10]:
# Level 2: Adding Detail
prompt = "Briefly describe the symptoms and treatments of diabetes."
assistant(prompt)

### Human: Briefly describe the symptoms and treatments of diabetes.
### Assistant :
Diabetes is a chronic condition that affects how your body regulates blood sugar levels. There are two main types of diabetes: Type 1 and Type 2.

**Type 1 Diabetes:**
Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the cells in the pancreas that produce insulin, a hormone that regulates blood sugar levels. Symptoms include increased thirst and urination, constant hunger, weight loss, blurred vision, and fatigue. Treatment involves taking insulin injections or using an insulin pump to replace the insulin that the body can't produce.

**Type 2 Diabetes:**
Type 2 diabetes is the most common form of diabetes, accounting for about 90-95% of all cases. It is characterized by insulin resistance, where the body's cells do not respond to insulin, and relative insulin deficiency. Symptoms include increased thirst and urination, constant hunger, weight gain, blurred v

In [11]:
# Level 3: Specify Information Type and Audience
prompt = "Explain the common symptoms and treatments of diabetes to a general audience."
assistant(prompt)

### Human: Explain the common symptoms and treatments of diabetes to a general audience.
### Assistant :
Diabetes is a chronic condition that affects how your body regulates blood sugar levels. There are two main types of diabetes: Type 1 and Type 2.

**Type 1 Diabetes:**
Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the cells in the pancreas that produce insulin, a hormone that regulates blood sugar levels. Without insulin, the body cannot use glucose for energy, leading to high blood sugar levels.

Common symptoms of Type 1 diabetes include:

* Frequent urination
* Increased thirst and hunger
* Fatigue
* Blurred vision
* Weight loss
* Slow-healing wounds

**Type 2 Diabetes:**
Type 2 diabetes is a metabolic disorder where the body becomes resistant to insulin or does not produce enough insulin. This leads to high blood sugar levels.

Common symptoms of Type 2 diabetes include:

* Frequent urination
* Increased thirst and hunger
* Fatigue


In [12]:
# Level 4: Request for Structured Response
prompt = "List the main symptoms of diabetes, then outline the standard treatments."
assistant(prompt)

### Human: List the main symptoms of diabetes, then outline the standard treatments.
### Assistant :
1. **Diabetes Symptoms:**
   - Increased thirst and hunger
   - Frequent urination
   - Fatigue
   - Blurred vision
   - Slow-healing sores
   - Frequent infections
   - Tingling or numbness in hands and feet

2. **Standard Treatments:**
   - **Lifestyle Changes:**
     - Healthy diet
     - Regular exercise
     - Weight management
     - Stress reduction
   - **Medications:**
     - Metformin (for type 2 diabetes)
     - Sulfonylureas (for type 2 diabetes)
     - Thiazolidinediones (for type 2 diabetes)
     - SGLT2 inhibitors (for type 2 diabetes)
     - GLP-1 receptor agonists (for type 2 diabetes)
     - Insulin injections (for type 1 diabetes)
   - **Insulin Therapy:**
     - Regular insulin
     - Rapid-acting insulin
     - Prandial insulin
     - Basal insulin
   - **Regular Check-ups:**
     - Blood glucose monitoring
     - HbA1c testing
     - Eye exams
     - Kidney functio

In [13]:
# Level 5: Differentiate by Type
prompt = "List the symptoms and treatments for both Type 1 and Type 2 diabetes."
assistant(prompt)

### Human: List the symptoms and treatments for both Type 1 and Type 2 diabetes.
### Assistant :
Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. Symptoms include increased thirst and urination, increased hunger, weight loss, blurred vision, and fatigue. Treatment involves insulin injections or an insulin pump, along with monitoring blood sugar levels and following a healthy diet and exercise regimen.

Type 2 diabetes is a metabolic disorder where the body becomes resistant to insulin or doesn't produce enough insulin. Symptoms include increased thirst and urination, increased hunger, weight loss, blurred vision, and fatigue. Treatment involves lifestyle changes such as diet and exercise, as well as oral medications or insulin injections.


In [14]:
# Level 6: Target a Professional Audience
prompt = "Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know."
assistant(prompt)

### Human: Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know.
### Assistant :

**Type 1 Diabetes**

* Symptoms: rapid weight loss, increased thirst and urination, blurred vision, fatigue, and frequent infections.
* Diagnostic criteria: HbA1c ≥ 8.5%, FPG ≥ 126 mg/dL, or random plasma glucose ≥ 200 mg/dL.
* Treatment options: insulin therapy, lifestyle modifications, and monitoring blood glucose levels.

**Type 2 Diabetes**

* Symptoms: similar to Type 1 diabetes, but may be milder.
* Diagnostic criteria: HbA1c ≤ 6.5%, FPG ≥ 126 mg/dL, or random plasma glucose ≥ 200 mg/dL.
* Treatment options: lifestyle modifications, oral medications, and insulin therapy.

**Key Points for Healthcare Providers**

* Both types of diabetes require ongoing monitoring and management to prevent complications.
* Type 1 diabetes requires insulin therapy, while Type 2 diabetes can be managed with lifestyle modifica

In [15]:
# Level 7: Clinical Detail with Disease Stages
prompt = "Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages."
assistant(prompt)

### Human: Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages.
### Assistant :
Diabetes is a chronic metabolic disorder characterized by high blood sugar levels. It can be classified into three main types: Type 1, Type 2, and prediabetes.

**Type 1 Diabetes:**
Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. This results in little to no insulin production. Symptoms include:
- Frequent urination
- Increased thirst and hunger
- Fatigue
- Blurred vision
- Slow healing of cuts and wounds
- Weight loss

Diagnosis involves:
- Fasting Plasma Glucose (FPG) test: 126 mg/dL or higher
- Oral Glucose Tolerance Test (OGTT): 200 mg/dL or higher
- Random Plasma Glucose: 200 mg/dL or higher

Treatment involves:
- Insulin therapy
- Lifestyle modifications (diet, exercise)



In [16]:
# Level 8: Add Nuance with Treatment Personalization
prompt = "Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors."
assistant(prompt)

### Human: Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors.
### Assistant :
Diabetes is a chronic condition characterized by high blood sugar levels. There are two main types: Type 1 and Type 2.

**Type 1 Diabetes**

Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. This results in little to no insulin production. Symptoms include:
- Frequent urination
- Increased thirst and hunger
- Fatigue
- Blurred vision
- Slow healing of cuts and wounds
- Weight loss

Diagnostic markers include:
- Fasting Plasma Glucose (FPG) test: 126 mg/dL or higher
- Oral Glucose Tolerance Test (OGTT): 200 mg/dL or higher
- Random Plasma Glucose test: 200 mg/dL or higher

Treatment options for Type 1 diabetes include:
- Insulin therapy: administered through

In [17]:
# Level 9: Consider Risks, Complications, and Treatment Impacts
prompt = "Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment."
assistant(prompt)

### Human: Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment.
### Assistant :
Diabetes is a chronic condition characterized by high blood sugar levels, which can be caused by insulin resistance, pancreatic beta-cell dysfunction, or a combination of both. The two main types of diabetes are Type 1 and Type 2.

**Type 1 Diabetes**

Type 1 diabetes is an autoimmune disease where the immune system attacks and destroys the insulin-producing beta cells in the pancreas. This results in little to no insulin production, requiring patients to take insulin injections or use an insulin pump to regulate blood sugar levels.

**Symptoms:**

- Frequent urination
- Increased thirst and hunger
- Fatigue
- Blurred visio

In [18]:
# Level 10: Advanced Detail for Clinical Research or Specialist Training
prompt = "Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes."
assistant(prompt)

### Human: Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes.
### Assistant :
Diabetes is a chronic metabolic disorder characterized by high blood glucose levels resulting from defects in insulin secretion, insulin action, or both. The two main types of diabetes are Type 1 and Type 2.

**Type 1 Diabetes**

Type 1 diabetes is an autoimmune disease in which the immune system attacks and destroys the insulin-producing beta cells in the pancreas. This results in little to no insulin production, requiring patients to take insulin injections or use an insulin pump to control